# Setup and run Bayesian inference on a batch of GW events

## Imports and setup

In [1]:
import os
import sys
import shutil
import glob
import h5py
import subprocess

import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex = True)

plt.rcParams.update(  # try to match font sizes of document
  {'axes.labelsize': 20,
   'axes.titlesize': 20,
   'legend.fontsize': 20,
   'xtick.labelsize': 20,
   'ytick.labelsize': 20,
   'text.usetex': True,
   'font.family': 'serif',
   'font.serif': ['palatino'],
   'savefig.dpi': 300
   })

%pylab inline

import GWNRTools

#sys.path.append('/home/prayush.kumar/src/GWNRTools/bin/')
#sys.path.append('/home/prayush/src/GWNRTools/bin/')
#sys.path.append('/home/prayush.kumar/local/venv/pycbc_master_enigma/src/GWNRTools/bin/')

Populating the interactive namespace from numpy and matplotlib


/home/prayush/local/venv/pycbc_inf/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['rc']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read


In [2]:
run_dir = '/home/prayush/research/test_pycbc_gw150914'
try: os.makedirs(run_dir)
except: pass
os.chdir(run_dir)

In [3]:
!pwd

/home/prayush/research/test_pycbc_gw150914


In [4]:
!rm -rf *

In [5]:
!ls

## Generate sampler / inference config files

In [6]:
!gwnrtools_write_inference_configs -h

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
usage: /home/prayush/src/GWNRTools/bin//gwnrtools_write_inference_configs [--options]

Get and write configuration files for generating a workflow to perform
Bayesian parameter estimation runs on a custom set of simulated signals

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --write-data-config WRITE_DATA_CONFIG
                        Write data config files and exit.
  --write-sampler-config WRITE_SAMPLER_CONFIG
                        Write sampler config files and exit.
  --write-inference-config WRITE_INFERENCE_CONFIG
                        Write inference config files and exit.
  --show-available-configs
                        Show available options for all configurations.
  --output-dir OUTPUT_DIR
                        

In [7]:
!gwnrtools_write_inference_configs --show-available-configs

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
inference: ['bbh_precessing']
data: ['gw150914-like-zeronoise', u'GW170608', u'GW170729', u'GW170817', u'GW151012', u'GW170818', u'GW170814', u'GW150914', u'GW170104', 'gw150914-like-gaussian', u'GW170809', u'GW151226', u'GW170823']
sampler: ['epsie', 'dynesty', 'multinest', 'emcee_pt', 'ultranest', 'emcee', 'cpnest']


In [8]:
!gwnrtools_write_inference_configs --verbose\
  --write-sampler-config emcee_pt --write-inference-config bbh_precessing

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
2020-03-04 19:11:19,386 Writing config file for sampler settings..
2020-03-04 19:11:19,386 Writing config file for inference settings..
2020-03-04 19:11:19,386 Done


In [9]:
!ls

bbh_precessing.ini  emcee_pt.ini


In [10]:
!cat emcee_pt.ini bbh_precessing.ini

[sampler]
name = emcee_pt
nwalkers = 500
ntemps = 20
;##### Other possible options
effective-nsamples = 4000
checkpoint-interval = 2000
max-samples-per-chain = 1000

[sampler-burn_in]
burn-in-test = nacl & max_posterior

;
;   Sampling transforms
;
[sampling_params]
; parameters on the left will be sampled in
; parametes on the right
mass1, mass2 : mchirp, q

[sampling_transforms-mchirp+q]
; inputs mass1, mass2
; outputs mchirp, q
name = mass1_mass2_to_mchirp_q
[model]
name = gaussian_noise
low-frequency-cutoff = 20.0

[variable_params]
; waveform parameters that will vary in MCMC
delta_tc =
mass1 =
mass2 =
spin1_a =
spin1_azimuthal =
spin1_polar =
spin2_a =
spin2_azimuthal =
spin2_polar =
distance =
coa_phase =
inclination =
polarization =
ra =
dec =

[static_params]
; waveform parameters that will not change in MCMC
approximant = IMRPhenomPv2
f_lower = 20
f_ref = 20
; we'll set the tc by using the trigger time in the data
; section of the config file + delta_tc
trigger_time = ${data|

## Write workflow config

In [11]:
out_str = '''\
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Executables
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[executables]
inference = ${which:pycbc_inference}
plot = ${which:pycbc_inference_plot_posterior}

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Workflow
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[workflow]
accounting-group = ligo.dev.o3.cbc.explore.test
log-path = log
sampler = emcee_pt.ini
inference = bbh_precessing.ini
events = GW150914 GW170104
sample-rate = 2048
data-sample-rate = 4096
data-duration = 4096

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Inference
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inference]
verbose =
seed = 12
config-files = inference.ini data.ini sampler.ini
output-file = inference.hdf
nprocesses = 10
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Visualize
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[plot]
input-file = inference.hdf
output-file = plots/posteriors.png
plot-scatter =
plot-marginal =
plot-prior = inference.ini data.ini
'''

with open('config.ini', 'w') as fout:
    fout.write(out_str)

In [12]:
!cat config.ini

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Executables
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[executables]
inference = ${which:pycbc_inference}
plot = ${which:pycbc_inference_plot_posterior}

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Workflow
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[workflow]
accounting-group = ligo.dev.o3.cbc.explore.test
log-path = log
sampler = emcee_pt.ini
inference = bbh_precessing.ini
events = GW150914 GW170104
sample-rate = 2048
data-sample-rate = 4096
data-duration = 4096

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Inference
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inference]
verbose =
seed = 12
config-files = inference.ini data.ini sampler.ini
output-file = inference.hdf
nprocesses = 10
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Visualize
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[plot]
input-file = inference.hdf
output-file = plots/posteriors.png
plot-scatter =
plot-marginal =
plot-prior = inference.ini data.ini


## Generate workflow

In [13]:
!gwnrtools_create_public_events_inference_workflow -h

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
usage: /home/prayush/src/GWNRTools/bin//gwnrtools_create_public_events_inference_workflow [--options]

Setup workflow to perform Bayesian parameter estimation runs on a custom set
of public gravitational-wave events using open data

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --output-dir OUTPUT_DIR
                        Output directory path.
  --force               If the output-dir already exists, overwrite it.
                        Otherwise, an OSError is raised.
  --save-backup         Don't delete the backup file after the run has
                        completed.
  --nprocesses NPROCESSES
                        Number of processes to use. If not given then only a
                        single core will be used.
  -

In [14]:
!gwnrtools_create_public_events_inference_workflow --config-files config.ini --output-dir . --force --verbose

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
2020-03-04 19:11:48,878 Using seed 0
2020-03-04 19:11:48,879 Will setup analyses in .
2020-03-04 19:11:48,879 Running with CPU support: 1 threads
2020-03-04 19:11:49,101 Reading configuration file
2020-03-04 19:11:49,103 Making workspace directories
2020-03-04 19:11:49,160 Creating DAG
2020-03-04 19:11:49,172 Making eventGW170104/emcee_pt/bbh_precessing in /home/prayush/research/test_pycbc_gw150914
2020-03-04 19:11:49,304 Copying config files to eventGW170104/emcee_pt/bbh_precessing
2020-03-04 19:11:49,305 Copying executables to eventGW170104/emcee_pt/bbh_precessing/scripts/
|==========================================| 129M/129M (100.00%)        25sETA     0s
|==========================================| 125M/125M (100.00%)        22sETA     0s
2020-03-04 19:12:39,789 Making eventGW150914/emcee_pt/bbh_precessing in /home/prayush/research/test_pycbc_gw150914
2020

In [15]:
!tree

.
├── bbh_precessing.ini
├── config.ini
├── emcee_pt.ini
├── eventGW150914
│   ├── data
│   │   ├── H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf
│   │   └── L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf
│   └── emcee_pt
│       └── bbh_precessing
│           ├── data.ini
│           ├── inference.ini
│           ├── log
│           ├── make_plot
│           ├── make_plot.sub
│           ├── plots
│           ├── run_inference
│           ├── run_inference.sub
│           ├── sampler.ini
│           └── scripts
│               ├── pycbc_inference
│               └── pycbc_inference_plot_posterior
├── eventGW170104
│   ├── data
│   │   ├── H-H1_GWOSC_4KHZ_R1-1167557889-4096.gwf
│   │   └── L-L1_GWOSC_4KHZ_R1-1167557889-4096.gwf
│   └── emcee_pt
│       └── bbh_precessing
│           ├── data.ini
│           ├── inference.ini
│           ├── log
│           ├── make_plot
│           ├── make_plot.sub
│           ├── plots
│           ├── run_inference
│           ├── run_inference.sub
│           ├── 

## Submit & monitor workflow

In [16]:
!condor_submit_dag pycbc_inference_events.dag >> dag.out

In [17]:
!tail *dagman.out

03/04/20 14:44:41 Workflow accounting_group_user: <>
03/04/20 14:44:41 Warning: failed to get attribute DAGNodeName
03/04/20 14:44:41 DAGMAN_LOG_ON_NFS_IS_ERROR setting: False
03/04/20 14:44:41 Default node log file is: </home/prayush/research/test_pycbc_gw150914/./pycbc_inference_events.dag.nodes.log>
03/04/20 14:44:41 DAG Lockfile will be written to pycbc_inference_events.dag.lock
03/04/20 14:44:41 DAG Input file is pycbc_inference_events.dag
03/04/20 14:44:41 Parsing 1 dagfiles
03/04/20 14:44:41 Parsing pycbc_inference_events.dag ...
03/04/20 14:44:41 Dag contains 4 total jobs
03/04/20 14:44:41 Sleeping for 3 seconds to ensure ProcessId uniqueness
